In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [ ]:
spark = (
  SparkSession
  .builder
  .appName("Spark SQL basic example")
  .getOrCreate()
  )

In [ ]:
columns = ["contrato", "fatura", "vlr_fat","valor_juros", "valor_multa", "fecham_fat"]
data = [
  (1, 12, 20000.35, 13.00, 10.00, '2022-04-28'),
  (1, 11, 1000.50, 13.00, 10.00, '2022-04-26'),
  (1, 10, 100.50, 13.00, 10.00, '2022-04-25'),
  (2, 22, 3000.43, 13.00, 10.00, '2022-04-29'),
  (2, 21, 1500.43, 13.00, 10.00, '2022-04-28'),
  (3, 21, -300.43, 0.0, 0.0, '2022-04-28'),
  (3, 22, -300.43, 0.0, 0.0, '2022-03-31'),
  (3, 20, -300.43, 0.0, 0.0, '2022-04-27'),
  ]
df = spark.createDataFrame(data,columns)

In [ ]:
scroll = (F.col('vlr_fat') + F.col('valor_juros') + F.col('valor_multa'))

In [ ]:
windowSpec  = Window.partitionBy("contrato").orderBy(F.desc('fecham_fat'))

In [ ]:
(
  df
  .withColumn("row",F.row_number().over(windowSpec))
  .show()
)

In [ ]:
windowSpecAgg  = Window.partitionBy("fecham_fat")

In [ ]:
(
  df
  .withColumn("row",F.row_number().over(windowSpec))
  .withColumn("soma_rolagem", scroll)
  .withColumn("max", F.max(F.col("fecham_fat")).over(windowSpecAgg))
  .show()
)

(
  df
  .withColumn("row",F.row_number().over(windowSpec))
  .withColumn("soma_rolagem", scroll)
  .withColumn("max", F.max(F.col("fecham_fat")).over(windowSpecAgg))
  .filter(F.col("row") == 1).select("*")
  .show()
)


In [ ]:
columns = ["contrato", "fatura", "vlr_fat","vlr_trans", "dt_trans", "fecham_fat"]
data = [
  (1, 10, 23.00, 23.00, '2022-03-20', '2022-04-05'),
  (1, 10, 34.00, 11.00, '2022-04-03', '2022-04-05'),
  (1, 11, 32.00, 32.00, '2022-04-06', '2022-05-05'),
  (1, 11, 72.00, 40.00, '2022-04-20', '2022-05-05'),
  (1, 11, 87.00, 15.00, '2022-05-02', '2022-05-05'),
  (1, 12, 15.00, 15.00, '2022-05-15', '2022-06-05'),
  (1, 12, 30.00, 15.00, '2022-06-02', '2022-06-05'),

  (2, 20, 23.00, 23.00, '2022-03-20', '2022-04-10'),
  (2, 20, 34.00, 11.00, '2022-04-08', '2022-04-10'),
  (2, 21, 32.00, 32.00, '2022-04-06', '2022-05-10'),
  (2, 21, 72.00, 40.00, '2022-04-20', '2022-05-10'),
  (2, 21, 87.00, 15.00, '2022-05-08', '2022-05-10'),
  (2, 22, 15.00, 15.00, '2022-05-15', '2022-06-10'),
  (2, 22, 30.00, 15.00, '2022-06-02', '2022-06-10'),

  (3, 30, 23.00, 23.00, '2022-03-20', '2022-04-15'),
  (3, 30, 34.00, 11.00, '2022-04-12', '2022-04-15'),
  (3, 31, 32.00, 32.00, '2022-04-06', '2022-05-15'),
  (3, 31, 72.00, 40.00, '2022-04-20', '2022-05-15'),
  (3, 31, 87.00, 15.00, '2022-05-12', '2022-05-15'),
  (3, 32, 15.00, 15.00, '2022-05-16', '2022-06-15'),
  (3, 32, 30.00, 15.00, '2022-06-02', '2022-06-15'),
  ]
df = spark.createDataFrame(data,columns)

In [ ]:
# sum values between date window pyspark
windowSpec = (
  Window.partitionBy('fatura')
  .orderBy(F.desc('fecham_fat'))
  .rowsBetween(
    Window.unboundedPreceding, Window.unboundedFollowing
    )
  )
windowSpec  = Window.partitionBy("contrato").orderBy(F.desc('fecham_fat'))


df.withColumn('sum_vlr_trans', F.sum(F.col('vlr_trans')).over(windowSpec)).show(50, False)

sum_trans = (F.sum(F.col('vlr_trans')))

(
  df.withColumn("row",F.row_number().over(windowSpec))
  .withColumn("sum_vlr_trans", sum_trans)
  .show()
)

(
  df
  .withColumn("row",F.row_number().over(windowSpec))
  .withColumn("sum_vlr_trans", sum_trans)
  .withColumn("max", F.max(F.col("fecham_fat")))
  .filter(F.col("row") == 1).select("*")
  .show()
)